In [1]:
import os
from pyspark import SparkContext, SparkConf

In [2]:
# sc = SparkContext(appName="Spark Apriori")
conf = SparkConf().setAppName("Spark Apriori").setMaster(os.getenv('SPARK_URL'))
sc = SparkContext(conf=conf)

23/04/27 16:54:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/27 16:55:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/27 16:55:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
sc

<SparkContext master=spark://cm041:46605 appName=Spark Apriori>

In [5]:
data = sc.textFile('./dataset/apriori_ratings.txt')

In [45]:
data.count()

163556

In [7]:
n_samples = data.count()
# min_sup to frequency
sup = n_samples * 0.2

In [12]:
itemset = data.map(lambda line: sorted([int(item) for item in line.strip().split(' ')]))
# [[858, 1246, 1968, 4226],
#  [605, 628], ...]

In [13]:
# braodcast vars cannot be rdd, so need to be collected 
shared_itemset = sc.broadcast(itemset.map(lambda x: set(x)).collect())

In [15]:
frequent_itemset = []

In [25]:
# prepare candidate_1
k = 1
c_k = itemset.flatMap(lambda x: set(x)).distinct().collect()

# candidate set
c_k = [{x} for x in c_k]

In [ ]:
def get_support(x): 
    

In [29]:
c_k_pr = sc.parallelize(c_k)

In [41]:
def get_support(x):
    x_sup = len([1 for t in shared_itemset.value if x.issubset(t)])
    if x_sup >= sup:
        return x, x_sup
    else:
        return ()

In [47]:
sc.parallelize(c_k).map(get_support).filter(lambda x: x).collect() # filter not None items 

[({858}, 45485), ({318}, 72655), ({608}, 38811)]

In [40]:
support.collect()[:10]

[4604, 4963, 9361, 7126, 9984, 45485, 72655, 19014, 11549, 12236]